![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/tutorials/quickstart-ci/AzureMLin10mins.png)

# Train and deploy a model in Azure Machine Learning


We will follow the following steps:
- Create the scripts for Data Extraction and converting it for model training and serving/prediction pipeline
- Create the script to build model and save the model and version control
- Create the script to use the saved model and do the prediction
- Create scripts to validate the models
- Create scripts to monitor the model performance
- Continuous Integration and Deployment scripts using open source platforms of your choice


In [1]:
import pandas as pd 
import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


##  Data

Dataset: https://www.kaggle.com/olistbr/marketing-funnel-olist


This dataset was generously provided by Olist, the largest department store in Brazilian marketplaces. Olist connects small businesses from all over Brazil to channels without hassle and with a single contract. Those merchants are able to sell their products through the Olist Store and ship them directly to the customers using Olist logistics partners. See more on our website: www.olist.com



In [2]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '71b2b08a-790d-402f-9800-fa9ad29fcb60'
resource_group = 'rg_sm'
workspace_name = 'SM1196'

workspace = Workspace(subscription_id, resource_group, workspace_name)




## Define functions for data preparation

In [3]:
#Load data
def load_data(path):
    df = Dataset.get_by_name(workspace, name=path)
    df = df.to_pandas_dataframe()

    return df


In [4]:
# Feature Engineering
# ------------------------------------------------
# flag = 0 for train data, flag = 1 for test data
# ------------------------------------------------

def feature_engineering(X, flag = 0, training_cols = None):

  X = X[['first_contact_date','origin','seller_id']]

  #Convert date to year, month, day, quarter
  X['year'] = pd.DatetimeIndex(X['first_contact_date']).year
  X['month'] = pd.DatetimeIndex(X['first_contact_date']).month
  X['day'] = pd.DatetimeIndex(X['first_contact_date']).day
  X['quarter'] = X['month'].apply(lambda x:x//4)

  #Drop contact date and seller id
  X.drop(columns=['first_contact_date','seller_id'], axis=1, inplace=True)

  X = pd.get_dummies(X, drop_first=True, prefix='', prefix_sep='')

  if flag == 1:
    X = X.T.reindex(training_cols).T.fillna(0)

  return X, X.columns


In [5]:
def prepare_train_validation_data(closed_deals_path, market_lead_path):

  closed_deals = load_data(closed_deals_path)
  market_lead = load_data(market_lead_path)
  
  #Join the two datasets
  mf = pd.merge(market_lead, closed_deals, left_on='mql_id', right_on='mql_id', how='left')

  # Create target variable
  mf['converted'] = mf[['seller_id']].where(mf[['seller_id']].isnull()==True, 1).fillna(0).astype(int)

  X = mf.loc[:, mf.columns != 'converted']
  y = mf.loc[:, ['converted']]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

  X_train, training_cols = feature_engineering(X_train)

  X_test, cols = feature_engineering(X_test, 1, training_cols)

  return X_train, X_test, y_train, y_test, training_cols


In [6]:
def evaluate(model, X, y):
  y_pred = model.predict(X.astype(np.int32))
  cm = confusion_matrix(y, y_pred)
  score = accuracy_score(y, y_pred)

  return y_pred, cm, score


In [7]:
closed_deals_path = 'olist_closed_deals' 
market_lead_path = 'olist_marketing_qualified_leads'

X_train, X_test, y_train, y_test, training_cols = prepare_train_validation_data(closed_deals_path, market_lead_path)

print("Done")


Done


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [8]:
def train_model(X, y, params):


  model = LogisticRegression(**params)
  model.fit(X, y)

  print(model)

  return model


## Train model and log metrics with MLflow

You'll train the model using the code below. Note that you are using MLflow autologging to track metrics and log model artefacts.

You'll be using the [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) classifier from the [SciKit Learn framework](https://scikit-learn.org/) to classify the data.

**Note: The model training takes approximately 2 minutes to complete.**

In [9]:
# create the model
import mlflow
import numpy as np
from sklearn.linear_model import LogisticRegression
from azureml.core import Workspace
from pprint import pprint

# connect to your workspace
ws = Workspace.from_config()

# create experiment and start logging to a new run in the experiment
experiment_name = "olist_v1"
model_name = "olist_sklearn_v1"

# set up MLflow to track the metrics
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)
mlflow.autolog()

def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

#params to train the model
params = {'max_iter':10,'random_state':0,'penalty': 'l1','solver':'liblinear'}

with mlflow.start_run() as run:
    model = train_model(X_train, y_train, params)
    
    params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)

    pprint(params)

    pprint(metrics)

    pprint(tags)

    pprint(artifacts)

print("Done")

2021/08/30 21:17:29 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2021/08/30 21:17:29 INFO mlflow.pyspark.ml: No SparkSession detected. Autologging will log pyspark.ml models contained in the default allowlist. To specify a custom allowlist, initialize a SparkSession prior to calling mlflow.pyspark.ml.autolog() and specify the path to your allowlist file via the spark.mlflow.pysparkml.autolog.logModelAllowlistFile conf.
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
2021/08/30 21:17:30 WARNING mlflow.utils.autologgin

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
{'C': '1.0',
 'class_weight': 'None',
 'dual': 'False',
 'fit_intercept': 'True',
 'intercept_scaling': '1',
 'l1_ratio': 'None',
 'max_iter': '10',
 'multi_class': 'auto',
 'n_jobs': 'None',
 'penalty': 'l1',
 'random_state': '0',
 'solver': 'liblinear',
 'tol': '0.0001',
 'verbose': '0',
 'warm_start': 'False'}
{'training_accuracy_score': 0.8906666666666667,
 'training_f1_score': 0.8391612599905971,
 'training_log_loss': 0.3252851163807794,
 'training_precision_score': 0.793287111111111,
 'training_recall_score': 0.8906666666666667,
 'training_roc_auc_score': 0.6777142646870893,
 'training_score': 0.8906666666666667}
{'estimator_class': 'sklearn.linear_model._logi


## Version control of our models with the model registry


In [10]:
import time

model_uri = "runs:/{}/model".format(run.info.run_id)

model = mlflow.register_model(
    model_uri,
    model_name
)
time.sleep(10)
current_version = model.version


Registered model 'olist_sklearn_v1' already exists. Creating a new version of this model...
2021/08/30 21:17:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: olist_sklearn_v1, version 2
Created version '2' of model 'olist_sklearn_v1'.


## Validate current model on test data

In [11]:
import mlflow
import mlflow.pyfunc
from sklearn.metrics import confusion_matrix, accuracy_score


model_version = current_version

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)


pprint(evaluate(model,X_test, y_test))



(array([0, 0, 0, ..., 0, 0, 0]),
 array([[1814,    0],
       [ 186,    0]]),
 0.907)


## Find the best model

In [12]:
import pandas as pd
import os
from mlflow.tracking.client import MlflowClient
from mlflow.entities import ViewType

experiment = MlflowClient().get_experiment_by_name(experiment_name)

# pprint(experiment)
experiment_ids = experiment.experiment_id

#Reading Pandas Dataframe from mlflow
runs= MlflowClient().search_runs(experiment_ids, run_view_type=ViewType.ALL)

score = None
run_id = 'NULL'
best_run = None
best_version = 0

client = MlflowClient()

for run in runs:
    if 'training_roc_auc_score' in run.data.metrics.keys():
        if(score == None or run.data.metrics['training_roc_auc_score'] > score):
            try:
                score = run.data.metrics['training_roc_auc_score']
                if len(client.search_model_versions(f"run_id ='{run.info.run_id}'"))>0:
                    best_run = run
                    run_id = run.info.run_id
 
            except:
                print("Metric not found for run_id:", run_id)

print("Best training roc_auc_score: ",score)
print("Run ID: ", run_id)

for mv in client.search_model_versions(f"run_id ='{run_id}'"):
    best_version = int(dict(mv)['version'])
    pprint(best_version)



Best training roc_auc_score:  0.6777142646870893
Run ID:  ee9745bd-78f0-40dd-b065-fcfd557cd45a
1


## Push the best version to staging

In [13]:
# Archive existing model in staging
max_version = 0
for mv in client.search_model_versions(f"name ='{model_name}'"):
    try:
        current_version = int(dict(mv)['version'])
        if current_version!= best_version and int(dict(mv)['current_stage']) == 'staging':
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="archived")
    except:
        print("Error with model")    

# Transition best model to staging
client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name=model_name,
    version=best_version,
    stage="staging")    


Error with model


<ModelVersion: creation_timestamp=1630354954381, current_stage='staging', description='', last_updated_timestamp=1630358267642, name='olist_sklearn_v1', run_id='ee9745bd-78f0-40dd-b065-fcfd557cd45a', run_link='', source='azureml://experiments/olist_v1/runs/ee9745bd-78f0-40dd-b065-fcfd557cd45a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

## Deploy the model for real-time inference
In this section you learn how to deploy a model so that an application can consume (inference) the model over REST.

### Create deployment configuration
The code cell gets a _curated environment_, which specifies all the dependencies required to host the model (for example, the packages like scikit-learn). Also, you create a _deployment configuration_, which specifies the amount of compute required to host the model. In this case, the compute will have 1CPU and 1GB memory.

In [14]:
# create environment for the deploy
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice

# get a curated environment
env = Environment.get(
    workspace=ws, 
    name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference",
    version=1
)
env.inferencing_stack_version='latest'

# create deployment config i.e. compute resources
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    tags={
            "data": "https://www.kaggle.com/olistbr/marketing-funnel-olist",
            "method": "predict"
        },
    description="Predict whether a lead will be converted or not",
)

### Deploy model

This next code cell deploys the model to Azure Container Instance (ACI).

**Note: The deployment takes approximately 3 minutes to complete.**

In [15]:
%%time
import uuid
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.model import Model

# get the registered model
model = Model(ws, model_name, version = best_version)

# create an inference config i.e. the scoring script and environment
inference_config = InferenceConfig(entry_script="score.py", environment=env)

pprint(inference_config)

# deploy the service
service_name = "sklearn-olist-lr-" + str(uuid.uuid4())[:4]
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
)

service.wait_for_deployment(show_output=True)

InferenceConfig(entry_script=score.py, runtime=None, conda_file=None, extra_docker_file_steps=None, source_directory=None, enable_gpu=None, base_image=None, base_image_registry=<azureml.core.container_registry.ContainerRegistry object at 0x7f697680ceb8>)
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-30 21:17:55+00:00 Creating Container Registry if not exists.
2021-08-30 21:17:55+00:00 Registering the environment.
2021-08-30 21:17:56+00:00 Use the existing image.
2021-08-30 21:17:56+00:00 Generating deployment configuration.
2021-08-30 21:17:57+00:00 Submitting deployment to compute.
2021-08-30 21:18:01+00:00 Checking the status of deployment sklearn-olist-lr-e0ff..
2021-08-30 21:19:09+00:00 Checking the status of inference endpoint sklearn-olist-lr-e0ff.
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: u

The [*scoring script*](score.py) file referenced in the code above can be found in the same folder as this notebook, and has two functions:

1. an `init` function that executes once when the service starts - in this function you normally get the model from the registry and set global variables
1. a `run(data)` function that executes each time a call is made to the service. In this function, you normally format the input data, run a prediction, and output the predicted result.

### View Endpoint
Once the model has been successfully deployed, you can view the endpoint by navigating to __Endpoints__ in the left-hand menu in Azure Machine Learning Studio. You will be able to see the state of the endpoint (healthy/unhealthy), logs, and consume (how applications can consume the model).

## Test the model service

You can test the model by sending a raw HTTP request to test the web service. 

In [16]:
# send raw HTTP request to test the web service.
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test) - 1)
input_data = '{"data": [' + str(list(X_test.iloc[random_index,])) + "]}"

print("Random_index: ", random_index)
pprint(input_data)

headers = {"Content-Type": "application/json"}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("label:", y_test.iloc[random_index,])
print("prediction:", resp.text)

Random_index:  1539
'{"data": [[2018, 2, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}'
POST to url http://dd392b2e-ab0f-44d8-a352-a8a74627d93b.eastus2.azurecontainer.io/score
label: converted    0
Name: 5587, dtype: int64
prediction: [0]


## Clean up resources

If you're not going to continue to use this model, delete the Model service using:

In [17]:
# Delete all web services
from azureml.core import Webservice
for webservice in Webservice.list(ws):
    print('name:', webservice.name)
    Webservice(ws, name = webservice.name).delete()


name: sklearn-olist-lr-e0ff
